# 원, 삼각형, 사각형 구분하기

### 3. Data Augmentation을 사용한 CNN 모델

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 부풀리기(Augmentation)

In [3]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.5,
    zoom_range=[0.8, 2.0],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [7]:
!mkdir preview

In [8]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 loop 주의
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='tri', save_format='png'):
  i += 1
  if i > 30:
    break

### 데이터 셋 생성하기

In [9]:
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


### 모델 정의/설정/학습

In [11]:
model = Sequential([
                    Conv2D(32, (3, 3), activation='relu', input_shape=(24, 24, 3)),
                    Conv2D(64, (3, 3), activation='relu'),
                    MaxPooling2D(pool_size=(2, 2)),
                    Flatten(),
                    Dense(128, activation='relu'),
                    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습
- 첫번째 인자: 훈련데이터셋 제공하는 제너레이터
- steps_per_epoch: 한 epoch에 사용한 스텝 수 지정
- epochs: 학습 반복 횟수
- validation_data: 검증데이터셋 제공하는 제너레이터
- validation_steps: 한 epoch 종료 시마다 검증할 때 사용되는 검증 스텝 수 지정

In [13]:
model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=200,
    validation_data=test_generator,
    validation_steps=5
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
15/15 [==============================] - 1s 37ms/step - loss: 1.1710 - accuracy: 0.3800 - val_loss: 1.0698 - val_accuracy: 0.6000
Epoch 2/200
15/15 [==============================] - 0s 24ms/step - loss: 1.0172 - accuracy: 0.5113 - val_loss: 1.0154 - val_accuracy: 0.5333
Epoch 3/200
15/15 [==============================] - 0s 23ms/step - loss: 0.8090 - accuracy: 0.6579 - val_loss: 0.9530 - val_accuracy: 0.7333
Epoch 4/200
15/15 [==============================] - 0s 22ms/step - loss: 0.4990 - accuracy: 0.7781 - val_loss: 1.5621 - val_accuracy: 0.4667
Epoch 5/200
15/15 [==============================] - 0s 24ms/step - loss: 0.9960 - accuracy: 0.5338 - val_loss: 1.3943 - val_accuracy: 0.6000
Epoch 6/200
15/15 [==============================] - 0s 23ms/step - loss: 0.6007 - accuracy: 0.8160 - val_loss: 1.2427 - val_accuracy: 0.4667
Epoch 7/200
15/15 [==============================] - 0s 24ms/step - loss: 0.5251 - accuracy: 0.7904 - val_loss: 1.1917 - val_accuracy: 0.6667
Epoch 

### 모델 평가

In [14]:
print('--Evaluation--')
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--Evaluation--
accuracy: 80.00%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
